In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('houses_balears.csv')
print("Filas originales:", len(df))

Filas originales: 24880


In [45]:
# Identificar valores no numéricos
non_numeric = df[~df['price'].astype(str).str.isnumeric()]
print("Valores no numéricos en price:", len(non_numeric))

# Convertir a numérico y eliminar inválidos
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df = df.dropna(subset=['price'])
print("\nFilas después de limpiar price:", len(df))

Valores no numéricos en price: 58

Filas después de limpiar price: 24822


In [46]:
display(df.head())

print("\nValores faltantes por columna:")
print(df.isnull().sum())

,ad_description,ad_last_update,air_conditioner,balcony,bath_num,built_in_wardrobe,chimney,condition,construct_date,energetic_certif,...,m2_useful,obtention_date,orientation,price,reduced_mobility,room_num,storage_room,swimming_pool,terrace,unfurnished
0,"Vendemos este precioso piso tipo duplex, estil...",Anuncio actualizado el 4 de abril,1,0,2,1,0,segunda mano/buen estado,NaN,en trámite,...,NaN,2019-04-08,sur,550000.0,0,2,0,1,1,NaN
1,"Piso en pleno centro de Santa Eulália, en buen...",más de 5 meses sin actualizar,1,0,1,1,0,segunda mano/buen estado,NaN,NaN,...,47,2019-04-08,norte,240000.0,0,1,0,0,0,NaN
2,Precioso piso en planta cuarta de un edificio ...,más de 5 meses sin actualizar,0,0,2,0,0,segunda mano/buen estado,NaN,no indicado,...,NaN,2019-04-08,NaN,775000.0,0,3,0,0,1,NaN
3,Completamente renovado por 1 año como arquitec...,más de 5 meses sin actualizar,0,0,1,1,0,segunda mano/buen estado,NaN,NaN,...,60,2019-04-08,NaN,300000.0,0,2,0,0,1,NaN
4,"En primera línea de mar, Dúplex de 144 m2, con...",más de 5 meses sin actualizar,1,1,2,1,0,segunda mano/buen estado,NaN,en trámite,...,144,2019-04-08,NaN,485000.0,0,3,0,1,1,NaN



Valores faltantes por columna:
ad_description         524
ad_last_update           0
air_conditioner          0
balcony                  0
bath_num                 0
built_in_wardrobe        0
chimney                  0
condition             6676
construct_date       14566
energetic_certif      6020
floor                 8265
garage               13793
garden                   0
ground_size          24822
heating              18531
house_id                 0
house_type               0
kitchen              24822
lift                 14720
loc_city                 0
loc_district          4633
loc_full                 0
loc_neigh            13200
loc_street           21373
loc_zone                 0
m2_real                  0
m2_useful            13176
obtention_date           0
orientation          14001
price                    0
reduced_mobility         0
room_num                 0
storage_room             0
swimming_pool            0
terrace                  0
unfurnished          24

In [47]:
columns_to_drop = [
    'ad_description',        # 524 nulos, texto libre (difícil de utilizar)
    'condition',             # 6676 nulos (66% faltante)
    'construct_date',        # 14566 nulos (77% faltante)
    'energetic_certif',      # 6020 nulos (60% faltante)
    'floor',                 # 8265 nulos (83% faltante)
    'garage',                # 13793 nulos (69% faltante)
    'ground_size',           # 24822 nulos (100% faltante)
    'heating',               # 18531 nulos (93% faltante)
    'house_id',              # Identificador único (no aporta al modelo)
    'kitchen',               # 24822 nulos (100% faltante)
    'lift',                  # 14720 nulos (74% faltante)
    'loc_district',          # 4633 nulos (46% faltante)
    'loc_neigh',             # 13200 nulos (66% faltante)
    'loc_street',            # 21373 nulos (100% faltante)
    'm2_useful',             # 13176 nulos (66% faltante, redundante con m2_real)
    'orientation',           # 14001 nulos (70% faltante)
    'unfurnished'            # 24822 nulos (100% faltante)
]

In [48]:
# Eliminar columnas seleccionadas
df_cleaned = df.drop(columns=columns_to_drop)

# Verificar resultado
print("\nColumnas restantes:", df_cleaned.columns.tolist())
print("\nValores faltantes después de limpieza:")
print(df_cleaned.isnull().sum())


Columnas restantes: ['ad_last_update', 'air_conditioner', 'balcony', 'bath_num', 'built_in_wardrobe', 'chimney', 'garden', 'house_type', 'loc_city', 'loc_full', 'loc_zone', 'm2_real', 'obtention_date', 'price', 'reduced_mobility', 'room_num', 'storage_room', 'swimming_pool', 'terrace']

Valores faltantes después de limpieza:
ad_last_update       0
air_conditioner      0
balcony              0
bath_num             0
built_in_wardrobe    0
chimney              0
garden               0
house_type           0
loc_city             0
loc_full             0
loc_zone             0
m2_real              0
obtention_date       0
price                0
reduced_mobility     0
room_num             0
storage_room         0
swimming_pool        0
terrace              0
dtype: int64
